In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from __future__ import print_function
import keras
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical
from scipy.interpolate import griddata
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
testdata = pd.read_csv('../input/Test.csv')
testdata = testdata.iloc[:,368:1392]
traindata = pd.read_csv('../input/Train.csv')
labels = traindata.iloc[:,-1]
traindata200 = traindata.iloc[:,0:200]
traindata = traindata.iloc[:,368:1392]

#traindata = traindata.interpolate(method ='cubic',axis=1)
#traindata['labels'] = pd.DataFrame(labels)
#traindata = traindata.iloc[:,368:1392]
# Any results you write to the current directory are saved as output.

In [ ]:
traindata200.shape

In [ ]:
x=0
dif = 0
for i in range(0,200):
    for j in range(0,1024):
        check = True
        for k in range(1,1000):
            if  ~np.isnan(traindata.iloc[k,j]) and ~np.isnan(traindata200.iloc[k,i]):
                if round(traindata.iloc[k,j]) > round(traindata200.iloc[k,i]) + 3 or round(traindata.iloc[k,j]) < round(traindata200.iloc[k,i]) - 3:
                    check = False
                    break

        if check == True:
            traindata.iloc[:,j] = traindata200.iloc[:,i]
            break
traindata.shape

In [ ]:
def format_to_lstm(df):
    X = df.values
    
    return np.reshape(X, (X.shape[0], 32, 32))


train_new = format_to_lstm(traindata)
print(train_new.shape)

In [ ]:


data_filled = []
for i in range(train_new.shape[0]):
    x = train_new[i,:,:]

    xx = np.arange(32)
    yy = np.arange(32)
    coords = np.array(np.meshgrid(xx,yy)).T.reshape(-1,2)
    mask = ~np.isnan(x)
    points = np.argwhere(mask)
    values = x[~np.isnan(x)]

    grid = griddata((points[:,0],points[:,1]), values, coords, method='cubic',fill_value=255)
    data_filled.append(grid)

In [ ]:
data_ready = data_filled


In [ ]:
data_ready = np.array(data_ready)
print(data_ready.shape)

In [ ]:

test_new = format_to_lstm(testdata)
test_new.shape

In [ ]:
test_filled = []

for i in range(test_new.shape[0]):
    x = test_new[i,:,:]

    xx = np.arange(32)
    yy = np.arange(32)
    coords = np.array(np.meshgrid(xx,yy)).T.reshape(-1,2)
    mask = ~np.isnan(x)
    points = np.argwhere(mask)
    values = x[~np.isnan(x)]

    grid = griddata((points[:,0],points[:,1]), values, coords, method='cubic',fill_value=255)
    test_filled.append(grid)
test_ready = np.array(test_filled)

In [ ]:
X_test = test_ready.reshape(test_ready.shape[0],32,32,1)

In [ ]:
X_train = np.array(data_ready)
X_train = X_train.reshape(X_train.shape[0],32,32,1)

In [ ]:
cnn_classifier = Sequential()

# 1st conv. layer
cnn_classifier.add(Conv2D(32, (3, 3), input_shape = (32, 32, 1), activation = 'relu'))
cnn_classifier.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd conv. layer
cnn_classifier.add(Conv2D(32, (3, 3), activation = 'relu')) 
cnn_classifier.add(MaxPooling2D(pool_size = (2, 2)))

# 3nd conv. layer
cnn_classifier.add(Conv2D(64, (3, 3), activation = 'relu')) 
cnn_classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
cnn_classifier.add(Flatten())

# Full connection
cnn_classifier.add(Dense(units = 64, activation = 'relu'))
cnn_classifier.add(Dropout(0.5)) 
cnn_classifier.add(Dense(units = 1, activation = 'sigmoid'))

cnn_classifier.compile(optimizer = 'adam', 
                       loss = 'binary_crossentropy', 
                       metrics = ['accuracy'])
 
cnn_classifier.fit(X_train, labels, batch_size=256, epochs=50)    


In [ ]:

reslt = cnn_classifier.predict_classes(X_test)
reslt

In [ ]:
id = np.arange(0,9700)
submission = pd.DataFrame({'id':id,'labels':reslt})
submission.shape

In [ ]:
filename = 'PredictionsF.csv'

submission.to_csv(filename,index=False)

#print('Saved file: ' + filename)

In [ ]:
# import numpy as np

# from keras.callbacks import EarlyStopping
# from keras.datasets import cifar10
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Flatten
# from keras.layers.convolutional import Conv2D
# from keras.optimizers import Adam
# from keras.layers.pooling import MaxPooling2D
# from keras.utils import to_categorical

# # For reproducibility
# np.random.seed(1000)

# if __name__ == '__main__':
#     # Load the dataset
#     X_train = np.array(data_ready)
#     X_train = X_train.reshape(X_train.shape[0],32,32,1)
#     Y_train = labels

#     # Create the model
#     model = Sequential()

#     model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 1)))
#     model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    

#     model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))

#     model.add(Flatten())
#     model.add(Dense(1024, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(10, activation='softmax'))

#     # Compile the model
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=Adam(lr=0.0001, decay=1e-6),
#                   metrics=['accuracy'])

#     # Train the model
#     model.fit(X_train / 255.0, to_categorical(Y_train),batch_size=256,shuffle=True,epochs=100,callbacks=[EarlyStopping(min_delta=0.001, patience=3)])

In [ ]:
#print(os.listdir("../output"))

In [ ]:
# grid = np.reshape(grid, (32, 32))


In [ ]:
# x= traindata.iloc[233]
# #x = x.fillna(x.median())
# x = x.values

# x= np.reshape(x,(32,32))


# xx = np.arange(32)
# yy = np.arange(32)
# coords = np.array(np.meshgrid(xx,yy)).T.reshape(-1,2)
# mask = ~np.isnan(x)
# points = np.argwhere(mask)
# values = x[~np.isnan(x)]

# grid = griddata((points[:,0],points[:,1]), values, coords, method='cubic',fill_value=255)
# grid= np.reshape(grid,(32,32))

# import matplotlib.pyplot as plt
# plt.imshow(grid, cmap='gray')
# plt.title('Cubic')
# plt.show()

In [ ]:

# X = x
# plt.imshow(X, cmap='gray')

In [ ]:
#traindata['labels'] = pd.DataFrame(labels)

In [ ]:
# td1 = traindata.loc[traindata['labels'] == 1.0]
# td2 = traindata.loc[traindata['labels'] == 2.0]
# td3 = traindata.loc[traindata['labels'] == 3.0]
# td4 = traindata.loc[traindata['labels'] == 4.0]
# td5 = traindata.loc[traindata['labels'] == 5.0]
# td6 = traindata.loc[traindata['labels'] == 6.0]
# td7 = traindata.loc[traindata['labels'] == 7.0]
# td8 = traindata.loc[traindata['labels'] == 8.0]
# td9 = traindata.loc[traindata['labels'] == 9.0] 
# td0 = traindata.loc[traindata['labels'] == 0.0]

In [ ]:
# df0 = (td0.std(axis=1))*3 - td0.mean(axis=1) 
# df0 = df0.loc[df0 < 0]
# df1 = (td1.std(axis=1))*3 - td1.mean(axis=1)
# df1 = df1.loc[df1 < 0]
# df2 = (td2.std(axis=1))*3 - td2.mean(axis=1)
# df2 = df2.loc[df2 < 0]
# df3 = (td3.std(axis=1))*3 - td3.mean(axis=1)
# df3 = df3.loc[df3 < 0]
# df4 = (td4.std(axis=1))*3 - td4.mean(axis=1)
# df4 = df4.loc[df4 < 0]
# df5 = (td5.std(axis=1))*3 - td5.mean(axis=1)
# df5 = df5.loc[df5 < 0]
# df6 = (td6.std(axis=1))*3 - td6.mean(axis=1)
# df6 = df6.loc[df6 < 0]
# df7 = (td7.std(axis=1))*3 - td7.mean(axis=1)
# df7 = df7.loc[df7 < 0]
# df8 = (td8.std(axis=1))*3 - td8.mean(axis=1)
# df8 = df8.loc[df8 < 0]
# df9 = (td9.std(axis=1))*3 - td9.mean(axis=1)
# df9 = df9.loc[df9 < 0]
# outliers = pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9])
# traindata = traindata.iloc[traindata.index[np.isin(traindata.index,outliers.index,invert=True)]]
# outliers

In [ ]:
# from __future__ import print_function
# import keras
# from keras.datasets import cifar10
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# import os

# batch_size = 256
# num_classes = 10
# epochs = 100
# data_augmentation = True
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'keras_cifar10_trained_model.h5'

# # The data, split between train and test sets:
# x_train = np.array(data_ready)
# x_train = x_train.reshape(x_train.shape[0],32,32,1)
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')


# # Convert class vectors to binary class matrices.
# y_train = keras.utils.to_categorical(labels, 10)

# #y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Conv2D(32, (3, 3), padding='same',input_shape=(32,32,1)))
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

# # initiate RMSprop optimizer
# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# # Let's train the model using RMSprop
# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=['accuracy'])

# x_train = x_train.astype('float32')

# x_train /= 255



# print('Not using data augmentation.')
# model.fit(x_train, y_train,
# batch_size=batch_size,
# epochs=epochs)

In [ ]:

# import matplotlib.pyplot as plt
# plt.imshow(grid)
# plt.title('Cubic')
# plt.show()